# Setup


In [1]:
import os
import cv2
import math
import pafy
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from moviepy.editor import *
from collections import deque
from enum import Enum
import csv
import matplotlib.pyplot as plt
 
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

seed_constant = 507
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

2023-12-03 16:48:56.877388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Grab, label, and normalize data

In [35]:
data_path = '../data/steps_in_3s_clean/'



class cat:
    front = 0
    heel = 1
    noise = 2
cat_count = [0,0,0]
test_input, train_input = [], []
test_labels, train_labels = [], []
test_input = np.array(test_input)
train_input = np.array(train_input)
input_count = 0
max_input = -1


for file_name in os.listdir(data_path):
    f_path = os.path.join(data_path, file_name)
    if os.path.isfile(f_path):
        f = open(f_path,'r')
        cat_str = f_path.split("ST")[1][0]
        cat_type = cat.noise
        speed = f_path.split("P")[1][0]
        if(cat_str == 'f'):
            cat_type = cat.front
        elif(cat_str == 'h'):
            cat_type = cat.heel
        row_num = 1
        rows = f.readlines()
        if not (cat_type == cat.noise):
            for r in rows:
                #print(r)
                triplet = np.abs(np.array(r.split(',')).astype(np.float64))
                print(np.size(triplet))
                if(np.size(triplet) == 99):
                    print("99 on row " + str(row_num) + " of " + f_path)
                    #print(triplet)
                row_num += 1
                input_count += 1
                cat_count[cat_type] += 1

                # check for new max
                temp_max = np.max(triplet)
                if(temp_max > max_input):
                    max_input = temp_max

                if speed == "4":
                    #test_input.append(triplet)
                    if(np.shape(test_input)[0] == 0):
                        test_input = triplet
                    else:
                        test_input = np.vstack([test_input,triplet])
                    test_labels.append(cat_type)
                else:
                    #train_input.append(triplet)
                    if(np.shape(train_input)[0] == 0):
                        train_input = triplet
                    else:
                        train_input = np.vstack([train_input,triplet])
                    train_labels.append(cat_type)
        f.close()

#normalize data
# train_input = train_input / max_input
# test_input = test_input / max_input

print("total input: ", input_count)
print("total test data: ", len(test_labels))
print("total training data: ", len(train_labels))
print("input by category (f,h,n): ", cat_count)

print(np.shape(np.array(train_labels)))

print(np.shape(train_input))

97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
97
99
99 on row 15 of ../data/steps_in_3s_clean/SP1_W175_STf_60s_steps.txt
[2.75000000e+02 5.75000000e+02 3.75000000e+02 4.82000000e+02
 5.10000000e+02 5.36000000e+02 8.16000000e+02 1.95000000e+02
 8.14000000e+02 1.18000000e+02 9.03000000e+02 1.43000000e+02
 5.44000000e+02 2.04000000e+02 5.09000000e+02 2.58000000e+02
 5.61000000e+02 6.58000000e+02 8.25000000e+02 7.67000000e+02
 6.84000000e+02 1.28800000e+03 2.00000000e+02 8.07000000e+02
 1.03000000e+02 3.47000000e+02 2.26000000e+02 1.00000000e+02
 3.92000000e+02 6.00000000e+01 3.97000000e+02 1.30000000e+01
 4.61000000e+02 1.20000000e+01 6.25000000e+02 8.10000000e+01
 5.28000000e+02 1.32000000e+02 5.81000000e+02 3.74000000e+02
 2.69000000e+02 6.88000000e+02 2.19000000e+02 4.40000000e

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 97 and the array at index 1 has size 99

# SVM


In [19]:
print(train_input)
#define Nonlinear SVM
from sklearn import svm
# clf = svm.NuSVC(gamma="auto")
# clf = svm.LinearSVC()
clf = svm.SVC()
clf.fit(train_input,np.array(train_labels))
test_pred = clf.predict(test_input)
print(np.sum(test_pred == test_labels)/len(test_labels))



[[array([9.17000000e+02, 8.45000000e+02, 8.27000000e+02, 9.75000000e+02,
         1.62000000e+02, 1.20800000e+03, 7.00000000e+01, 1.43700000e+03,
         1.14000000e+02, 1.21300000e+03, 4.80000000e+01, 7.91000000e+02,
         1.52000000e+02, 8.94000000e+02, 1.58000000e+02, 8.52000000e+02,
         1.70000000e+02, 8.00000000e+02, 1.11000000e+02, 7.68000000e+02,
         7.90000000e+01, 5.86000000e+02, 2.46000000e+02, 6.70000000e+02,
         2.18000000e+02, 4.61000000e+02, 3.89000000e+02, 5.86000000e+02,
         1.92000000e+02, 2.38000000e+02, 2.74000000e+02, 7.10000000e+01,
         4.57000000e+02, 9.00000000e+00, 3.61000000e+02, 2.88000000e+02,
         2.48000000e+02, 4.43000000e+02, 2.58000000e+02, 4.95000000e+02,
         1.36000000e+02, 4.66000000e+02, 3.50000000e+01, 4.99000000e+02,
         2.10000000e+02, 1.00000000e+02, 5.13000000e+02, 6.17000000e+02,
         8.75000000e+02, 6.85000000e+02, 1.35700000e+03, 1.07900000e+03,
         8.10000000e+02, 1.37800000e+03, 5.31000000

ValueError: setting an array element with a sequence.